In [1]:
#x01_populationStatistics

In [2]:
from chewc.callback import *
from chewc.policy import *
from chewc.sim import *
from chewc.lab import *
from chewc.config import *

In [3]:
import matplotlib.pyplot as plt
import uuid
import pdb
import torch
from matplotlib.animation import FuncAnimation
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import gymnasium as gym
import numpy as np
device='cpu'
from torch.utils.tensorboard import SummaryWriter



import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CallbackList


In [4]:
config = get_default_config()
for i in config:
    print(f"{i} : {config[i]}")
env = create_simulation()


action_low : 0.05
action_high : 0.95
sparse_reward : False
n_parents : 10
n_chr : 1
n_loci : 100
pop_size : 200
max_generations : 10
h2 : 0.5
target_mean : 0
target_variance : 1
reps : 1
total_timesteps : 100000
learning_rate : 0.0003
gae_lambda : 0.95
log_freq : 100
start_gen : 10
end_gen : 100
start_gae_lambda : 0.9
end_gae_lambda : 0.95
seed : None


In [6]:
env.reset()

({'population': array([[[[0, 0, 1, ..., 0, 0, 0]],
  
          [[0, 0, 1, ..., 0, 0, 0]]],
  
  
         [[[1, 0, 1, ..., 1, 1, 1]],
  
          [[0, 0, 1, ..., 0, 0, 0]]],
  
  
         [[[0, 0, 0, ..., 0, 0, 0]],
  
          [[0, 0, 0, ..., 0, 0, 0]]],
  
  
         ...,
  
  
         [[[0, 1, 0, ..., 0, 1, 0]],
  
          [[0, 0, 0, ..., 0, 1, 0]]],
  
  
         [[[0, 0, 1, ..., 0, 0, 0]],
  
          [[0, 0, 1, ..., 0, 0, 0]]],
  
  
         [[[0, 0, 0, ..., 0, 1, 0]],
  
          [[0, 1, 1, ..., 0, 0, 0]]]], dtype=int32),
  'generation': array([0.], dtype=float32)},
 {'max_phenotype': 4.192286491394043,
  'genetic_variance': 1.0000001192092896,
  'current_generation': 0})

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def collect_baselines(env, actions, repetitions=10, cycles=5):
    results = {action: {'max_phenotype': [], 'gv': []} for action in actions}
    final_gen_averages = {}
    
    for action in actions:
        final_gen_phenotypes = []
        for _ in range(repetitions):
            env.reset()
            cycle_max_phenotype = []
            cycle_gv = []
            for _ in range(cycles):
                env.step(np.array([action]))
                max_phenotype = env.population.breeding_values.max()
                gv = env.population.breeding_values.var()
                cycle_max_phenotype.append(max_phenotype)
                cycle_gv.append(gv)
            
            results[action]['max_phenotype'].append(cycle_max_phenotype)
            results[action]['gv'].append(cycle_gv)
            final_gen_phenotypes.append(cycle_max_phenotype[-1])
        
        final_gen_averages[action] = np.mean(final_gen_phenotypes)
    
    best_action = max(final_gen_averages, key=final_gen_averages.get)
    best_average = final_gen_averages[best_action]
    
    return results, best_action, best_average

def plot_best_run(results, best_action):
    plt.figure(figsize=(10, 6))
    
    max_phenotypes = np.array(results[best_action]['max_phenotype'])
    mean_values = np.mean(max_phenotypes, axis=0)
    std_values = np.std(max_phenotypes, axis=0)
    cycles = range(len(mean_values))
    
    plt.plot(cycles, mean_values, label=f'Action {best_action:.3f}')
    plt.fill_between(cycles, mean_values - std_values, mean_values + std_values, alpha=0.3)
    
    plt.xlabel('Cycle')
    plt.ylabel('Max Phenotype')
    plt.title(f'Max Phenotype over Cycles for Best Action')
    plt.legend()
    plt.grid(True)
    plt.show()

# Usage
actions = np.linspace(-1, 0, 10)  # or however many actions you want to test
results, best_action, best_average = collect_baselines(env, actions, repetitions=10, cycles=env.SP.max_generations)

print(f"Best action: {best_action:.3f}")
print(f"Best average max phenotype in final generation: {best_average:.3f}")

# Plot only the best run
plot_best_run(results, best_action)

In [ ]:
#np linear ( between -1 and 1 )




In [ ]:

# Create your custom callbacks
genetic_variance_callback = AverageFinalGenerationCallback(log_freq=100)
action_callback = ActionTrackingCallback(log_freq=10)

# Combine the callbacks using CallbackList
combined_callbacks = CallbackList([genetic_variance_callback, action_callback])
vec_env = DummyVecEnv([lambda: env])
# Create and train the model with the custom policy
model = PPO(CustomActorCriticPolicy, vec_env, verbose=1, tensorboard_log="./ppotb")
model.learn(total_timesteps=config['total_timesteps'], callback=combined_callbacks)

In [ ]:
# Assuming you have an environment 'env' properly wrapped in a VecEnv
obs = vec_env.reset()
done = False

while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    print(scale_values(action))
    print(reward)
    # Process the observation, reward, and info as needed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO  # Assuming you're using PPO

def collect_baselines(env, actions, repetitions=10, cycles=5):
    results = {action: {'max_phenotype': [], 'gv': []} for action in actions}
    final_gen_averages = {}
    
    for action in actions:
        final_gen_phenotypes = []
        for _ in range(repetitions):
            env.reset()
            cycle_max_phenotype = []
            cycle_gv = []
            for _ in range(cycles):
                env.step(np.array([action]))
                max_phenotype = env.population.breeding_values.max()
                gv = env.population.breeding_values.var()
                cycle_max_phenotype.append(max_phenotype)
                cycle_gv.append(gv)
            
            results[action]['max_phenotype'].append(cycle_max_phenotype)
            results[action]['gv'].append(cycle_gv)
            final_gen_phenotypes.append(cycle_max_phenotype[-1])
        
        final_gen_averages[action] = np.mean(final_gen_phenotypes)
    
    best_action = max(final_gen_averages, key=final_gen_averages.get)
    best_average = final_gen_averages[best_action]
    
    return results, best_action, best_average

def run_model(model, vec_env, num_runs=100):
    all_max_phenotypes = []
    
    for _ in range(num_runs):
        obs = vec_env.reset()
        done = False
        episode_max_phenotypes = []
        
        while not done:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = vec_env.step(action)
            max_phenotype = vec_env.get_attr('population')[0].breeding_values.max()
            episode_max_phenotypes.append(max_phenotype)
        
        all_max_phenotypes.append(episode_max_phenotypes)
    
    return np.array(all_max_phenotypes)

def plot_comparison(baseline_results, best_action, model_results):
    plt.figure(figsize=(12, 6))
    
    # Plot baseline
    baseline_max_phenotypes = np.array(baseline_results[best_action]['max_phenotype'])
    baseline_mean = np.mean(baseline_max_phenotypes, axis=0)
    baseline_std = np.std(baseline_max_phenotypes, axis=0)
    cycles = range(len(baseline_mean))
    
    plt.plot(cycles, baseline_mean, label=f'Baseline (Action {best_action:.3f})', color='blue')
    plt.fill_between(cycles, baseline_mean - baseline_std, baseline_mean + baseline_std, alpha=0.3, color='blue')
    
    # Plot model results
    model_mean = np.mean(model_results, axis=0)
    model_std = np.std(model_results, axis=0)
    
    plt.plot(cycles, model_mean, label='Model', color='red')
    plt.fill_between(cycles, model_mean - model_std, model_mean + model_std, alpha=0.3, color='red')
    
    plt.xlabel('Cycle/Generation')
    plt.ylabel('Max Phenotype')
    plt.title('Comparison of Baseline vs Model: Max Phenotype over Cycles')
    plt.legend()
    plt.grid(True)
    plt.show()

# Usage
actions = np.linspace(-1, 0, 20)
baseline_results, best_action, best_average = collect_baselines(env, actions, repetitions=30, cycles=env.SP.max_generations)

print(f"Best baseline action: {best_action:.3f}")
print(f"Best baseline average max phenotype in final generation: {best_average:.3f}")

# Load your trained model
# model = PPO.load("path_to_your_model")

# Run the model
model_results = run_model(model, vec_env, num_runs=100)

# Plot comparison
plot_comparison(baseline_results, best_action, model_results)

In [ ]:
model_results

In [ ]:
torch.mean(env.population.breeding_values)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Convert torch tensor to numpy array if necessary
phenotypes = env.population.breeding_values.numpy()

# Calculate summary statistics
mean_phenotype = np.mean(phenotypes)
median_phenotype = np.median(phenotypes)
std_phenotype = np.std(phenotypes)
min_phenotype = np.min(phenotypes)
max_phenotype = np.max(phenotypes)
sum_phenotype = np.sum(phenotypes)

# Print summary statistics
print("Summary Statistics:")
print(f"Mean: {mean_phenotype:.2f}")
print(f"Median: {median_phenotype:.2f}")
print(f"Standard Deviation: {std_phenotype:.2f}")
print(f"Minimum: {min_phenotype:.2f}")
print(f"Maximum: {max_phenotype:.2f}")
print(f"Sum: {sum_phenotype:.2f}")

# Create histogram
plt.figure(figsize=(10, 6))
plt.hist(phenotypes, bins=10, edgecolor='black')
plt.title('Histogram of Phenotypes')
plt.xlabel('Phenotype Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()